In [1]:
import nltk
import re
import numpy as np
import pandas as pd
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy
import pyLDAvis
import pyLDAvis.gensim
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings("ignore",category=DeprecationWarning)
import pickle
from nltk.corpus import stopwords
stop_words = stopwords.words('english')
from nltk.stem import WordNetLemmatizer

C:\ProgramData\Anaconda3\lib\site-packages\past\types\oldstr.py:33: DeprecationWarning: invalid escape sequence \d
  """


In [3]:
data = pd.read_csv('abcnews-date-text.csv', error_bad_lines =False)
data['publish_date'] = pd.to_datetime(data['publish_date'].astype(str), format = '%Y%m%d')

In [6]:
data['publish_date'] = pd.DatetimeIndex(data['publish_date']).year

In [7]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data['headline_text']))

In [8]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [9]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)
data_words_nostops = pd.Series(data_words_nostops)
data_lemmatized = data_words_nostops.apply(lambda x: [WordNetLemmatizer().lemmatize(word, pos='v') for word in x])

In [10]:
temp = data
temp['lemmatize'] = data_lemmatized
for i in range(15):
    globals()['trend{}'.format(i+2003)] = temp.loc[temp.publish_date == i+2003]

In [11]:
trend_list = [trend2003, trend2004, trend2005, trend2006, trend2007, trend2008, trend2009, trend2010, trend2011, trend2012, trend2013, trend2014, trend2015, trend2016, trend2017]

In [12]:
lemmatized = list(trend2015['lemmatize'])
id2word = corpora.Dictionary(lemmatized)
texts = lemmatized
corpus = [id2word.doc2bow(text) for text in texts]

In [13]:
# Build LDA model
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=20, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [15]:
# Compute Perplexity
print('\nPerplexity: ', lda_model.log_perplexity(corpus))  # a measure of how good the model is. lower the better.

# Compute Coherence Score
coherence_model_lda = CoherenceModel(model=lda_model, texts=lemmatized, dictionary=id2word, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Perplexity:  -25.65348551907308

Coherence Score:  0.4240479090789065


In [14]:
# Print the Keyword in the 10 topics
pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.162*"australian" + 0.146*"call" + 0.109*"melbourne" + 0.077*"price" + '
  '0.051*"western" + 0.046*"high" + 0.044*"victoria" + 0.043*"four" + '
  '0.033*"prepare" + 0.031*"girl"'),
 (1,
  '0.125*"attack" + 0.112*"back" + 0.108*"mine" + 0.062*"deal" + 0.059*"fan" + '
  '0.057*"celebrate" + 0.049*"target" + 0.041*"hill" + 0.037*"party" + '
  '0.033*"reveal"'),
 (2,
  '0.290*"fire" + 0.136*"nsw" + 0.130*"hobart" + 0.114*"rural" + 0.072*"world" '
  '+ 0.036*"boat" + 0.032*"turn" + 0.013*"flight" + 0.011*"around" + '
  '0.006*"well"'),
 (3,
  '0.285*"country" + 0.256*"hour" + 0.100*"podcast" + 0.033*"leaders" + '
  '0.030*"thursday" + 0.019*"abbott" + 0.018*"tony" + 0.014*"policy" + '
  '0.012*"agricultural" + 0.009*"shorten"'),
 (4,
  '0.307*"new" + 0.220*"year" + 0.076*"force" + 0.074*"fall" + 0.063*"dead" + '
  '0.044*"tas" + 0.028*"farmer" + 0.017*"fruit" + 0.008*"philippines" + '
  '0.007*"injury"'),
 (5,
  '0.286*"qld" + 0.182*"release" + 0.002*"paper" + 0.000*"north" + '
  